<a href="https://colab.research.google.com/github/zeloso21/models/blob/master/valueatrisk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from tiingo import TiingoClient
import pandas as pd, datetime, numpy as np
import matplotlib.pyplot as plt
import datetime
import warnings
warnings.filterwarnings('ignore') #Tiingo API is returning a warning due to an upcoming pandas update
#User Set Up
data = {'Stocks':['GOOGL', 'TSLA','AAPL'], 'Quantity':[100, 50, 300]} #Define your holdings
ScenariosNo = 500 #Define the number of scenarios you want to run
Percentile = 99 #Define your confidence interval
VarDaysHorizon = 1 #Define your time period
info = 1 #1 if you want more info returned by the script
# Create a DataFrame of holdings
df = pd.DataFrame(data)
print('[INFO] Calculating the max amount of money the portfolio will lose within', VarDaysHorizon, 'days', Percentile, 'percent of the time.')
today = datetime.date.today() - datetime.timedelta(days=1)
def is_business_day(date):
    return bool(len(pd.bdate_range(date, date)))
def dateforNoOfScenarios(date):
    i=0
    w=0
    while i < ScenariosNo:
        if (is_business_day(today - datetime.timedelta(days = w)) == True):
            i = i+1
            w = w+1
        else:
            w = w+1
            continue
    #print('gotta go back these many business days',i)
    #print('gotta go back these many days',w)
    #remember to add an extra day (days +1 = scenario numbers)
    return(today - datetime.timedelta(days = w*1.04 + 1)) #4% is an arbitary number i've calculated the holidays to be in 500days.
def SourceHistoricPrices():
    if info == 1: print('[INFO] Fetching stock prices for portfolio holdings')
    #Set Up for Tiingo
    config = {}
    config['session'] = True
    config['api_key'] = '2763b8e218f691f1c6a56c23959fa8a6f0e1e922' #tiingo에서 private key를 받음, 회원가입시
    client = TiingoClient(config)
    #Create a list of tickers for the API call
    Tickers = []
    i=0
    for ticker in data:
        while i <= len(data):
            Tickers.append(data[ticker][i])
            i=i+1
    if info == 1: print('[INFO] Portfolio Holdings determined as', Tickers)
    if info == 1: print('[INFO] Portfolio Weights determined as', data['Quantity'])
    #Call the API and store the data
    global HistData
    HistData = client.get_dataframe(Tickers, metric_name='close', startDate=dateforNoOfScenarios(today), endDate=today)
    if info == 1: print('[INFO] Fetching stock prices completed.', len(HistData), 'days.')
    return(HistData)

[INFO] Calculating the max amount of money the portfolio will lose within 1 days 99 percent of the time.


In [13]:
def ValuePortfolio():
    HistData['PortValue'] = 0
    i=0
    if info == 1: print('[INFO] Calculating the portfolio value for each day')
    while i<= len(data):
        stock = data['Stocks'][i]
        quantity = data['Quantity'][i]
        HistData['PortValue'] = HistData[stock] * quantity + HistData['PortValue']
        i = i+1
def CalculateVaR():
    if info == 1: print('[INFO] Calculating Daily % Changes')
    #calculating percentage change
    HistData['Perc_Change'] = HistData['PortValue'].pct_change()
    #calculate money change based on current valuation
    HistData['DollarChange'] = HistData.loc[HistData.index.max()]['PortValue'] * HistData['Perc_Change'] 
    if info == 1: print('[INFO] Picking', round(HistData.loc[HistData.index.max()]['PortValue'],2),' value from ', HistData.index.max().strftime('%Y-%m-%d'), ' as the latest valuation to base the monetary returns')
    ValueLocForPercentile = round(len(HistData) * (1 - (Percentile / 100)))
    if info == 1: print('[INFO] Picking the', ValueLocForPercentile, 'th highest value')
    global SortedHistData
    SortedHistData = HistData.sort_values(by=['DollarChange'])
    if info == 1: print('[INFO] Sorting the results by highest max loss')
    VaR_Result = SortedHistData.iloc[ValueLocForPercentile + 1,len(SortedHistData.columns)-1] * np.sqrt(VarDaysHorizon)
    print('The portfolio\'s VaR is:', round(VaR_Result,2))
def CalculateES():
    ValueLocForPercentile = round(len(HistData) * (1 - (Percentile / 100)))
    ES_Result = round(SortedHistData['DollarChange'].head(ValueLocForPercentile).mean(axis=0),2) * np.sqrt(VarDaysHorizon)
    print('The portfolios\'s Expected Shortfall is', ES_Result)


In [15]:
SourceHistoricPrices()
ValuePortfolio()
CalculateVaR()
CalculateES()


[INFO] Fetching stock prices for portfolio holdings
[INFO] Portfolio Holdings determined as ['GOOGL', 'TSLA', 'AAPL']
[INFO] Portfolio Weights determined as [100, 50, 300]
[INFO] Fetching stock prices completed. 503 days.
[INFO] Calculating the portfolio value for each day
[INFO] Calculating Daily % Changes
[INFO] Picking 309414.5  value from  2020-07-01  as the latest valuation to base the monetary returns
[INFO] Picking the 5 th highest value
[INFO] Sorting the results by highest max loss
The portfolio's VaR is: -15272.52
The portfolios's Expected Shortfall is -26220.17
